In [1]:
%autosave 0

Autosave disabled


In [2]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0


[notice] A new release of pip is available: 24.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install keras-image-helper

In [3]:
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model-v4.h5

--2025-09-12 13:31:12--  https://github.com/DataTalksClub/machine-learning-zoomcamp/releases/download/chapter7-model/xception_v4_large_08_0.894.h5
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/256401220/0156a400-0049-11eb-8490-c0d01b48ea8c?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-12T12%3A17%3A09Z&rscd=attachment%3B+filename%3Dxception_v4_large_08_0.894.h5&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-12T11%3A16%3A56Z&ske=2025-09-12T12%3A17%3A09Z&sks=b&skv=2018-11-09&sig=y41BMiK73ncPNZuR8mwF6ro5bLxlY6DDxtQJi5pVcbI%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NzY3Njk3MywibmJmIjoxNz

In [4]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model("clothing-model-v4.h5")
tf.saved_model.save(model, "clothing-model")

2025-09-12 13:31:20.944579: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-12 13:31:21.759371: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /tmp/.mount_cursorjVcT9F/usr/lib/:/tmp/.mount_cursorjVcT9F/usr/lib32/:/tmp/.mount_cursorjVcT9F/usr/lib64/:/tmp/.mount_cursorjVcT9F/lib/:/tmp/.mount_cursorjVcT9F/lib/i386-linux-gnu/:/tmp/.mount_cursorjVcT9F/lib/x86_64-linux-gnu/:/tmp/.mount_cursorjVcT9F/lib/aarch64-linux-gnu/:/tmp/.mount_cursorjVcT9F/lib32/:/tmp/.mount_cursorjVcT9F/lib64/:
2025-09-12 13:31:21.759399: I tensorflow/compiler/xla/st

INFO:tensorflow:Assets written to: clothing-model/assets


INFO:tensorflow:Assets written to: clothing-model/assets


In [5]:
!tree clothing-model

clothing-model
├── assets
├── fingerprint.pb
├── saved_model.pb
└── variables
    ├── variables.data-00000-of-00001
    └── variables.index

2 directories, 4 files


In [6]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [ ]:
host = 'localhost:8500'# port where our tf-service is currently running (see above docker run)

channel = grpc.insecure_channel(host)
# access this port (using insecure as tf-service will not be accessible from outside of Kubernetes)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
# tf-serving

In [11]:
from keras_image_helper import create_preprocessor

In [13]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [14]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [15]:
X

array([[[[-0.11372548, -0.15294117, -0.19999999],
         [-0.11372548, -0.15294117, -0.19999999],
         [-0.10588235, -0.14509803, -0.19215685],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.11372548, -0.11372548, -0.17647058]],

        [[-0.09019607, -0.12941176, -0.17647058],
         [-0.09019607, -0.12941176, -0.17647058],
         [-0.08235294, -0.12156862, -0.16862744],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.10588235, -0.10588235, -0.16862744]],

        [[-0.09803921, -0.1372549 , -0.18431371],
         [-0.09803921, -0.1372549 , -0.18431371],
         [-0.09019607, -0.12941176, -0.17647058],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.03529412, -0.03529412, -0.09803921],
         [-0.09019607, -0.09019607, -0.15294117]],

        ...,

        [[-0.67058825, -0.7019608 , -0

In [16]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [17]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [18]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [20]:
pb_response

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.87986421585083
    float_val: -4.75631046295166
    float_val: -2.359531879425049
    float_val: -1.08926522731781
    float_val: 9.903782844543457
    float_val: -2.826179027557373
    float_val: -3.648310422897339
    float_val: 3.241154909133911
    float_val: -2.612096071243286
    float_val: -4.852035045623779
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [21]:
preds = pb_response.outputs['dense_7'].float_val

In [22]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [23]:
dict(zip(classes, preds))

{'dress': -1.87986421585083,
 'hat': -4.75631046295166,
 'longsleeve': -2.359531879425049,
 'outwear': -1.08926522731781,
 'pants': 9.903782844543457,
 'shirt': -2.826179027557373,
 'shoes': -3.648310422897339,
 'shorts': 3.241154909133911,
 'skirt': -2.612096071243286,
 't-shirt': -4.852035045623779}